# Agenda

1. Lots of questions
2. Even more questions!

# What are the methods that all Python objects have?

- Everything in Python is an object
- Everything has a class (type)
- Everything has attributes as well (things that come after a .)

- Methods are attributes
- Through inheritance, objects have access to methods on their classes, and also anything that their class inherits from

- Everyone eventually inherits from `object`, the ultimate top object in the Python hierarchy.

So the only methods that *everything* in Python has access to are those that are defined on `object`, and which we then 